In [1]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split as tts
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import psutil
import glob
import os

Using TensorFlow backend.


In [2]:
def getMinData(dir):
    minValue = 9999999
    for file in os.listdir(dir):
        fileValue = np.load(dir + file).shape[0]
        if minValue > fileValue:
            minValue = fileValue
    return minValue

# defining global variabals

In [3]:
# All images are 28 * 28 * 1 = 784
# 1 because images are grey scalded images
IMAGE_ROWS = 28
IMAGE_COLS = 28
IMAGE_SIZE = IMAGE_ROWS * IMAGE_ROWS

# Path to data
IMAGE_DIR = "./data/"

# Total number of numpy file
NUM_IMAGES = len(glob.glob1(IMAGE_DIR, "*.npy"))

# How many element in each image
TOTAL_ELEMENTS_PER_IMAGE = 500

MAX_NUM_ELEMENTS = getMinData(IMAGE_DIR)

NUM_ITERATON = MAX_NUM_ELEMENTS // TOTAL_ELEMENTS_PER_IMAGE

BATCH_SIZE = 512
IMAGE_SHAPE = (IMAGE_ROWS, IMAGE_COLS, 1)

# setting for all labels a unique value
label_map = {}

for i, file in enumerate(os.listdir(IMAGE_DIR)):
    label_map[file[:-4]] = i

# Creating the Model
## Defining the model

In [4]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=3,
                 activation="relu", input_shape=IMAGE_SHAPE))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(Conv2D(filters=64, kernel_size=3,
                 activation="relu"))
model.add(Dropout(0.25))
model.add(Conv2D(filters=128, kernel_size=3,
                 activation="relu"))
model.add(Dropout(0.3))
model.add(Conv2D(filters=256, kernel_size=3,
                 activation="relu"))
model.add(Dropout(0.35))
model.add(Conv2D(filters=512, kernel_size=3,
                 activation="relu"))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(NUM_IMAGES, activation="relu"))
model.add(Dense(NUM_IMAGES, activation="relu"))
model.add(Dense(NUM_IMAGES, activation="softmax"))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
dropout_2 (Dropout)          (None, 11, 11, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 9, 9, 128)         73856     
_________________________________________________________________
dropout_3 (Dropout)          (None, 9, 9, 128)         0         
__________

## Compiling the model

In [5]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])

# Loading the data

In [6]:
for i in range(0, NUM_ITERATON):
    print("{} of {}".format(i, NUM_ITERATON))
    minBound = i * TOTAL_ELEMENTS_PER_IMAGE
    maxBound = minBound + TOTAL_ELEMENTS_PER_IMAGE

    x_train = np.ndarray(shape=(0, 784), dtype=np.float32)
    y_train = np.array([], dtype=np.uint16)

    for file in os.listdir(IMAGE_DIR):
        elements = np.load(IMAGE_DIR + file)[minBound:maxBound] / 255
        x_train = np.concatenate((x_train, elements))
        labels = np.full(elements.shape[0], label_map[file[:-4]])
        y_train = np.append(y_train, labels)

    # Spliting the loaded data into testing and validating
    x_train, x_validate, y_train, y_validate = tts(x_train, y_train, test_size=.2)

    # Reshape the data

    x_train = x_train.reshape(x_train.shape[0], *IMAGE_SHAPE)
    x_validate = x_validate.reshape(x_validate.shape[0], *IMAGE_SHAPE)

    # Fiting the model

    model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=128, verbose=1,
              validation_data=(x_validate, y_validate))

    # Saving the model after every fit
    model.save('model_first.h5')

0 of 227
Train on 138000 samples, validate on 34500 samples
Epoch 1/128
138000/138000 [==============================] - 36s 259us/step - loss: 4.8513 - acc: 0.0718 - val_loss: 3.8212 - val_acc: 0.1832
Epoch 2/128
138000/138000 [==============================] - 27s 195us/step - loss: 3.3472 - acc: 0.2644 - val_loss: 2.8451 - val_acc: 0.3553
Epoch 3/128
138000/138000 [==============================] - 27s 196us/step - loss: 2.7158 - acc: 0.3771 - val_loss: 2.4937 - val_acc: 0.4237
Epoch 4/128
138000/138000 [==============================] - 27s 196us/step - loss: 2.4107 - acc: 0.4331 - val_loss: 2.3125 - val_acc: 0.4599
Epoch 5/128
138000/138000 [==============================] - 27s 196us/step - loss: 2.2200 - acc: 0.4710 - val_loss: 2.1694 - val_acc: 0.4898
Epoch 6/128
138000/138000 [==============================] - 27s 196us/step - loss: 2.0803 - acc: 0.4992 - val_loss: 2.0825 - val_acc: 0.5100
Epoch 7/128
138000/138000 [==============================] - 27s 195us/step - loss: 1.97

Epoch 58/128
138000/138000 [==============================] - 25s 181us/step - loss: 0.8315 - acc: 0.7495 - val_loss: 2.3516 - val_acc: 0.5514
Epoch 59/128
138000/138000 [==============================] - 25s 182us/step - loss: 0.8277 - acc: 0.7515 - val_loss: 2.3819 - val_acc: 0.5553
Epoch 60/128
138000/138000 [==============================] - 25s 181us/step - loss: 0.8205 - acc: 0.7531 - val_loss: 2.3978 - val_acc: 0.5540
Epoch 61/128
138000/138000 [==============================] - 27s 193us/step - loss: 0.8145 - acc: 0.7539 - val_loss: 2.3988 - val_acc: 0.5495
Epoch 62/128
138000/138000 [==============================] - 27s 193us/step - loss: 0.8021 - acc: 0.7576 - val_loss: 2.4110 - val_acc: 0.5563
Epoch 63/128
138000/138000 [==============================] - 27s 194us/step - loss: 0.7961 - acc: 0.7598 - val_loss: 2.4102 - val_acc: 0.5535
Epoch 64/128
138000/138000 [==============================] - 27s 193us/step - loss: 0.7902 - acc: 0.7618 - val_loss: 2.4380 - val_acc: 0.5526

Epoch 44/128
138000/138000 [==============================] - 25s 184us/step - loss: 0.8275 - acc: 0.7556 - val_loss: 2.2739 - val_acc: 0.5762
Epoch 45/128
138000/138000 [==============================] - 25s 184us/step - loss: 0.8240 - acc: 0.7550 - val_loss: 2.2852 - val_acc: 0.5714
Epoch 46/128
138000/138000 [==============================] - 26s 186us/step - loss: 0.8101 - acc: 0.7592 - val_loss: 2.2983 - val_acc: 0.5717
Epoch 47/128
138000/138000 [==============================] - 26s 185us/step - loss: 0.8010 - acc: 0.7602 - val_loss: 2.3046 - val_acc: 0.5717
Epoch 48/128
138000/138000 [==============================] - 26s 186us/step - loss: 0.7975 - acc: 0.7625 - val_loss: 2.3205 - val_acc: 0.5734
Epoch 49/128
138000/138000 [==============================] - 26s 186us/step - loss: 0.7875 - acc: 0.7646 - val_loss: 2.3309 - val_acc: 0.5706
Epoch 50/128
138000/138000 [==============================] - 26s 186us/step - loss: 0.7855 - acc: 0.7647 - val_loss: 2.3686 - val_acc: 0.5704

Epoch 27/128
138000/138000 [==============================] - 25s 181us/step - loss: 1.0592 - acc: 0.7060 - val_loss: 1.9217 - val_acc: 0.5975
Epoch 28/128
138000/138000 [==============================] - 25s 182us/step - loss: 1.0401 - acc: 0.7092 - val_loss: 1.9270 - val_acc: 0.5962
Epoch 29/128
138000/138000 [==============================] - 25s 181us/step - loss: 1.0181 - acc: 0.7139 - val_loss: 1.9533 - val_acc: 0.5961
Epoch 30/128
138000/138000 [==============================] - 25s 182us/step - loss: 1.0071 - acc: 0.7151 - val_loss: 1.9644 - val_acc: 0.5968
Epoch 31/128
138000/138000 [==============================] - 25s 182us/step - loss: 0.9884 - acc: 0.7200 - val_loss: 1.9869 - val_acc: 0.5921
Epoch 32/128
138000/138000 [==============================] - 25s 182us/step - loss: 0.9739 - acc: 0.7221 - val_loss: 1.9989 - val_acc: 0.5930
Epoch 33/128
138000/138000 [==============================] - 25s 182us/step - loss: 0.9605 - acc: 0.7250 - val_loss: 2.0141 - val_acc: 0.5956

Epoch 13/128
138000/138000 [==============================] - 25s 181us/step - loss: 1.4153 - acc: 0.6377 - val_loss: 1.7060 - val_acc: 0.6109
Epoch 14/128
138000/138000 [==============================] - 25s 181us/step - loss: 1.3792 - acc: 0.6454 - val_loss: 1.6997 - val_acc: 0.6102
Epoch 15/128
138000/138000 [==============================] - 25s 182us/step - loss: 1.3570 - acc: 0.6492 - val_loss: 1.7173 - val_acc: 0.6085
Epoch 16/128
138000/138000 [==============================] - 25s 181us/step - loss: 1.3258 - acc: 0.6547 - val_loss: 1.7270 - val_acc: 0.6083
Epoch 17/128
138000/138000 [==============================] - 25s 181us/step - loss: 1.2975 - acc: 0.6593 - val_loss: 1.7374 - val_acc: 0.6101
Epoch 18/128
138000/138000 [==============================] - 25s 181us/step - loss: 1.2689 - acc: 0.6647 - val_loss: 1.7534 - val_acc: 0.6077
Epoch 19/128
138000/138000 [==============================] - 25s 181us/step - loss: 1.2512 - acc: 0.6668 - val_loss: 1.7777 - val_acc: 0.6053

...

138000/138000 [==============================] - 29s 208us/step - loss: 0.9323 - acc: 0.7494 - val_loss: 1.9086 - val_acc: 0.6163
Epoch 118/128
138000/138000 [==============================] - 28s 202us/step - loss: 0.9335 - acc: 0.7480 - val_loss: 1.8818 - val_acc: 0.6198- acc: 0.74 - ETA: 1s - loss: 0.9314
Epoch 119/128
138000/138000 [==============================] - 29s 210us/step - loss: 0.9380 - acc: 0.7477 - val_loss: 1.9146 - val_acc: 0.6161
Epoch 120/128
138000/138000 [==============================] - 29s 209us/step - loss: 0.9350 - acc: 0.7483 - val_loss: 1.9113 - val_acc: 0.6139 - loss: 0.9334 -
Epoch 121/128
138000/138000 [==============================] - 29s 209us/step - loss: 0.9266 - acc: 0.7503 - val_loss: 1.9340 - val_acc: 0.6154
Epoch 122/128
138000/138000 [==============================] - 28s 205us/step - loss: 0.9176 - acc: 0.7514 - val_loss: 1.9058 - val_acc: 0.6178
Epoch 123/128
138000/138000 [==============================] - 29s 207us/step - loss: 0.9220 - ac

138000/138000 [==============================] - 40s 293us/step - loss: 0.9845 - acc: 0.7371 - val_loss: 1.8460 - val_acc: 0.6250
Epoch 100/128
138000/138000 [==============================] - 42s 302us/step - loss: 0.9856 - acc: 0.7365 - val_loss: 1.8366 - val_acc: 0.6220
Epoch 101/128
138000/138000 [==============================] - 41s 300us/step - loss: 0.9898 - acc: 0.7361 - val_loss: 1.8276 - val_acc: 0.6223
Epoch 102/128
138000/138000 [==============================] - 41s 300us/step - loss: 0.9813 - acc: 0.7384 - val_loss: 1.8542 - val_acc: 0.6234
Epoch 103/128
138000/138000 [==============================] - 41s 298us/step - loss: 0.9841 - acc: 0.7376 - val_loss: 1.8406 - val_acc: 0.6231
Epoch 104/128
138000/138000 [==============================] - 42s 303us/step - loss: 1.0091 - acc: 0.7325 - val_loss: 1.8553 - val_acc: 0.6237
Epoch 105/128
138000/138000 [==============================] - 42s 304us/step - loss: 0.9685 - acc: 0.7390 - val_loss: 1.8846 - val_acc: 0.6222
Epoch 

Epoch 28/128
138000/138000 [==============================] - 42s 306us/step - loss: 1.3508 - acc: 0.6616 - val_loss: 1.5999 - val_acc: 0.6354
Epoch 29/128
138000/138000 [==============================] - 43s 308us/step - loss: 1.3421 - acc: 0.6626 - val_loss: 1.5991 - val_acc: 0.6338
Epoch 30/128
138000/138000 [==============================] - 42s 306us/step - loss: 1.3349 - acc: 0.6637 - val_loss: 1.5991 - val_acc: 0.6387
Epoch 31/128
138000/138000 [==============================] - 41s 296us/step - loss: 1.3328 - acc: 0.6634 - val_loss: 1.6087 - val_acc: 0.6372
Epoch 32/128
138000/138000 [==============================] - 42s 307us/step - loss: 1.3245 - acc: 0.6648 - val_loss: 1.5980 - val_acc: 0.6376
Epoch 33/128
138000/138000 [==============================] - 42s 303us/step - loss: 1.3269 - acc: 0.6661 - val_loss: 1.6048 - val_acc: 0.6341
Epoch 34/128
138000/138000 [==============================] - 45s 327us/step - loss: 1.3134 - acc: 0.6675 - val_loss: 1.6133 - val_acc: 0.6335

Epoch 14/128
138000/138000 [==============================] - 25s 180us/step - loss: 1.5023 - acc: 0.6293 - val_loss: 1.5788 - val_acc: 0.6337
Epoch 15/128
138000/138000 [==============================] - 25s 180us/step - loss: 1.4938 - acc: 0.6300 - val_loss: 1.5696 - val_acc: 0.6362
Epoch 16/128
138000/138000 [==============================] - 25s 184us/step - loss: 1.4687 - acc: 0.6368 - val_loss: 1.5987 - val_acc: 0.6302
Epoch 17/128
 82432/138000 [================>.............] - ETA: 9s - loss: 1.4539 - acc: 0.6371

KeyboardInterrupt: 